##**Machine Learning Language Translation**

In [2]:
# Import necessary Libraries
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

In [3]:
batch_size = 64 # Batch size for training.
epochs = 100 # Number of epochs to train for.
latent_dim = 256 # Latent dimensionality of the encoding space.
num_samples = 10000 # Number of samples to train on.

# Path to the data text file on disk.
data_path = '/content/eng-fra.txt'

In [4]:
# Check a few lines from the file
with open('/content/eng-fra.txt', 'r', encoding='utf-8') as f:
    for i in range(5):
        print(f.readline())


Go.	Va !

Run!	Cours !

Run!	Courez !

Wow!	Ça alors !

Fire!	Au feu !



In [5]:
# Vectorize the data
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

# Load and process the data
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().strip().split('\n')

for line in lines[:min(num_samples, len(lines))]:
    parts = line.split('\t')

    # Only process lines with exactly two parts: input and target
    if len(parts) == 2:
        input_text, target_text = parts

        # Add start (\t) and end (\n) tokens to target text
        target_text = '\t' + target_text + '\n'

        input_texts.append(input_text)
        target_texts.append(target_text)

        # Update character sets
        input_characters.update(set(input_text))
        target_characters.update(set(target_text))

# Sort characters to ensure consistent indexing
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

In [6]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [7]:
# Print stats
print("Number of samples:", len(input_texts))
print("Unique input characters:", len(input_characters))
print("Unique target characters:", len(target_characters))
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for targets:", max_decoder_seq_length)
print("Example input:", input_texts[0])
print("Example target:", target_texts[0])

Number of samples: 10000
Unique input characters: 71
Unique target characters: 91
Max sequence length for inputs: 16
Max sequence length for targets: 59
Example input: Go.
Example target: 	Va !



In [8]:
input_tokens = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_tokens = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [9]:
input_tokens, target_tokens

({' ': 0,
  '!': 1,
  '$': 2,
  '&': 3,
  "'": 4,
  ',': 5,
  '-': 6,
  '.': 7,
  '0': 8,
  '1': 9,
  '2': 10,
  '3': 11,
  '4': 12,
  '5': 13,
  '6': 14,
  '7': 15,
  '9': 16,
  ':': 17,
  '?': 18,
  'A': 19,
  'B': 20,
  'C': 21,
  'D': 22,
  'E': 23,
  'F': 24,
  'G': 25,
  'H': 26,
  'I': 27,
  'J': 28,
  'K': 29,
  'L': 30,
  'M': 31,
  'N': 32,
  'O': 33,
  'P': 34,
  'Q': 35,
  'R': 36,
  'S': 37,
  'T': 38,
  'U': 39,
  'V': 40,
  'W': 41,
  'Y': 42,
  'Z': 43,
  'a': 44,
  'b': 45,
  'c': 46,
  'd': 47,
  'e': 48,
  'f': 49,
  'g': 50,
  'h': 51,
  'i': 52,
  'j': 53,
  'k': 54,
  'l': 55,
  'm': 56,
  'n': 57,
  'o': 58,
  'p': 59,
  'q': 60,
  'r': 61,
  's': 62,
  't': 63,
  'u': 64,
  'v': 65,
  'w': 66,
  'x': 67,
  'y': 68,
  'z': 69,
  '’': 70},
 {'\t': 0,
  '\n': 1,
  ' ': 2,
  '!': 3,
  '$': 4,
  '&': 5,
  "'": 6,
  ',': 7,
  '-': 8,
  '.': 9,
  '0': 10,
  '1': 11,
  '5': 12,
  '6': 13,
  '9': 14,
  ':': 15,
  '?': 16,
  'A': 17,
  'B': 18,
  'C': 19,
  'D': 20,
  'E'

In [10]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [11]:
# One Hot representation
for i in range(len(input_texts)):
    for t, char in enumerate(input_texts[i]):
        encoder_input_data[i, t, input_tokens[char]] = 1
    encoder_input_data[i, t + 1:, input_tokens[' ']] = 1
    for t, char in enumerate(target_texts[i]):
      # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_tokens[char]] = 1
        if t > 0:
          # decoder_target_data will be ahead by one timestep
          # and will not include the starter character
            decoder_target_data[i, t - 1, target_tokens[char]] = 1
    decoder_input_data[i, t + 1:, target_tokens[' ']] = 1
    decoder_target_data[i, t:, target_tokens[' ']] = 1


In [12]:
encoder_input_data[0].shape

(16, 71)

**Creation of LSTM layer**

In [13]:
# Define an input sequence and process it
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# we discard "encoder_outputs" and only keep the states
encoder_states = [state_h, state_c]

In [14]:
# Setup the decoder, using "encoder_states" as initial state
decoder_inputs = Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [15]:
# Define the model that will turn
# "encoder_input_data" & "decoder_input_data" into "decoder_target_data"
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.6841 - loss: 1.6483 - val_accuracy: 0.6892 - val_loss: 1.1821
Epoch 2/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7251 - loss: 1.0263 - val_accuracy: 0.7043 - val_loss: 1.0752
Epoch 3/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7460 - loss: 0.9142 - val_accuracy: 0.7233 - val_loss: 0.9643
Epoch 4/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7739 - loss: 0.8093 - val_accuracy: 0.7581 - val_loss: 0.8511
Epoch 5/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.7954 - loss: 0.7137 - val_accuracy: 0.7661 - val_loss: 0.8146
Epoch 6/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8051 - loss: 0.6700 - val_accuracy: 0.7725 - val_loss: 0.7752
Epoch 7/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.8135 - loss: 0.6373 - val_accuracy: 0.7788 - val_loss: 0.7465
Epoch 8/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8203 - loss: 0.6118 - 

In [19]:
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# Fix: correct variable name from `state` to `state_c`
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]  # Fix applied here

decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-Lookup token index to encode sequences back to something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_tokens.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_tokens.items())

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    state_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_tokens['\t']] = 1.0

    decoded_sentence = ''
    stop_condition = False

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + state_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length or find stop character.
        if (sampled_char == '\n' or
            len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        state_value = [h, c]

    return decoded_sentence

# Test decoding
for seq_index in range(10):  # Reduce to 10 to avoid large output
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
-
Input sentence: Go.
Decoded sentence: Prends tout !

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
-
Input sentence: Run!
Decoded 

#####**Explanation of sampling model**

In [20]:
# Define sampling models

# This model will be used during inference to encode the input sentence to a state vector.
encoder_model = Model(encoder_inputs, encoder_states)

# Define inputs for the decoder's LSTM hidden and cell states during inference
decoder_state_input_h = Input(shape=(latent_dim,))  # hidden state
decoder_state_input_c = Input(shape=(latent_dim,))  # cell state
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]  # list of both state inputs

# Reuse the decoder LSTM layer and feed the previous hidden and cell states into it
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)

# Capture the decoder's final states to feed back into the next time step
decoder_states = [state_h, state_c]

# Apply the final dense (softmax) layer to generate a probability distribution over the target vocabulary
decoder_outputs = decoder_dense(decoder_outputs)

# Define the final decoder model used for inference
# It takes decoder input tokens and the initial decoder states, and returns the output tokens and new states
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Create a reverse lookup dictionary to convert token indices back to readable characters (input side)
reverse_input_char_index = dict(
    (i, char) for char, i in input_tokens.items())

# Same as above but for the output (target) vocabulary
reverse_target_char_index = dict(
    (i, char) for char, i in target_tokens.items())


###**decode_sequence() Function Explanation**

In [21]:
def decode_sequence(input_seq):
    # Encode the input sequence into state vectors using the encoder model
    state_value = encoder_model.predict(input_seq)

    # Create an empty target sequence of shape (1 batch, 1 time-step, num_decoder_tokens)
    target_seq = np.zeros((1, 1, num_decoder_tokens))

    # Initialize the first character of the target sequence with the start-of-sequence token '\t'
    target_seq[0, 0, target_tokens['\t']] = 1.0

    decoded_sentence = ''  # Store the final translated sentence here
    stop_condition = False  # Flag to stop the loop

    while not stop_condition:
        # Predict the next character token and new states using the decoder model
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + state_value)

        # Choose the most likely next token using argmax
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]

        # Append the predicted character to the output sentence
        decoded_sentence += sampled_char

        # Check if the end-of-sequence character is generated or the sentence is too long
        if (sampled_char == '\n' or
            len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Prepare the next input token (one-hot vector) for the decoder
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update the decoder states to use in the next iteration
        state_value = [h, c]

    return decoded_sentence


####**Testing the Decoder on the First 10 Samples**

In [22]:
# Loop through the first 10 samples to test the model's translation capability
for seq_index in range(10):
    # Prepare one input sequence from the training data
    input_seq = encoder_input_data[seq_index: seq_index + 1]

    # Generate the translated output sequence
    decoded_sentence = decode_sequence(input_seq)

    # Print input and translated output
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
-
Input sentence: Go.
Decoded sentence: Prends tout !

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
-
Input sentence: Run!
Decoded s

In [18]:
# Next: infernce mode (sampling)
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# Use the same decoder_lstm layer defined earlier
# Use distinct variable names for outputs
decoder_outputs_lstm_inference, state_h_inference, state_c_inference = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)

# Use the same decoder_dense layer defined earlier
decoder_outputs_inference = decoder_dense(decoder_outputs_lstm_inference)

# Define the decoder model for inference
# It takes the decoder input sequence and the previous states
# It outputs the predicted next token probabilities and the new states
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs_inference, state_h_inference, state_c_inference]) # Output the new states explicitly

# Reverse-Lookup token index to encode sequences back to something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_tokens.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_tokens.items())


def decode_sequence(input_seq):
  # Encode the input as state vectors.
  state_value = encoder_model.predict(input_seq)

  # Generate empty target sequence of length 1.
  target_seq = np.zeros((1, 1, num_decoder_tokens))
  # Populate the first character of target sequence with the start character.
  target_seq[0, 0, target_tokens['\t']] = 1.

  # Sampling loop for a batch of sequences
  #  (to simplify, here we assume a batch of size 1).
  stop_condition = False
  decoded_sentence = ''
  # Keep track of current states
  current_states = state_value # Initialize with states from encoder

  while not stop_condition:
    output_tokens, h, c = decoder_model.predict(
        [target_seq] + current_states) # Use current_states for prediction

    # Sample a token
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = reverse_target_char_index[sampled_token_index]
    decoded_sentence += sampled_char

    # Exit condition: either hit max length or find stop character.
    if (sampled_char == '\n' or
        len(decoded_sentence) > max_decoder_seq_length):
        stop_condition = True

    # Update the target sequence (of length 1).
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, sampled_token_index] = 1.

    # Update states for the next iteration
    current_states = [h, c]

  return decoded_sentence


for seq_index in range(100):
  # Take one sequence (part of the training set)
  # for trying out decoding
  input_seq = encoder_input_data[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)
  print('-')
  print('Input sentence:', input_texts[seq_index])
  print('Decoded sentence:', decoded_sentence)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
-
Input sentence: Go.
Decoded sentence: Prends tout !

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
-
Input sentence: Run!
Decoded s